In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_categorical
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, make_scorer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import lightgbm as lgm
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import scipy
from scipy.cluster import hierarchy as hc

In [3]:
# used functionality 

def mad(x,y): return sum(abs(x-y))/len(x)

def handle_multi_string_columns(df, column, single_strings):
    '''
    Replaces column whose fields contain several strings with new columns. Each
    new column will then represent a single string
    
    INPUT:
    df - the pandas dataframe you want to search
    column - the column name you want to look through
    single_strings - a list of strings you want to search for in each row of df[col]

    OUTPUT:
    new_df - The dataframe without the multi-string column but with the newly created columns
    col_dict - Dictionary translating names of the new columns to their corresponding string
    '''
    
    #collects new columns of indicating if a certain index refers to a string 
    new_columns = dict()
    
    #dict column name -> string name
    col_dict = dict()
    
    #loop through list of strings
    counter = 0
    for string in single_strings:
        bool_list = []
        #loop through rows
        for idx in range(df.shape[0]):
            #if the ed type is in the row set to True
            if string in str(df[column][idx]):
                bool_list.append(1)
            else:
                bool_list.append(0)
        col_name = column + "_" + str(counter)
        new_columns[col_name] = bool_list
        col_dict[col_name] = string
        counter = counter + 1
    
    new_df = df.drop(column,axis=1)
    
    new_df = pd.concat([new_df, pd.DataFrame(data=new_columns, index = df.index, dtype=int)], axis=1)
    
    return new_df, col_dict


def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}).sort_values('imp', ascending=False)

def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

def mean_absolute_error_2(y_true, y_pred):
    return mean_absolute_error(y_true, np.round(y_pred))


def classification(y_train, y_test_preds):
    y_train_min = np.min(y_train)
    y_train_max = np.max(y_train)
    total_distance = y_train_max - y_train_min
    point_0 = y_train_min + total_distance * np.sum(y_train == 0) / len(y_train)
    point_1 = point_0 + total_distance * np.sum(y_train == 1) / len(y_train)
    point_2 = point_1 + total_distance * np.sum(y_train == 2) / len(y_train)
    point_3 = point_2 + total_distance * np.sum(y_train == 3) / len(y_train)
    point_4 = y_train_max
    
    for i in range(len(y_test_preds)):
        if y_test_preds[i] <= point_0:
            y_test_preds[i] = 0
        elif y_test_preds[i] <= point_1:
            y_test_preds[i] = 1
        elif y_test_preds[i] <= point_2:
            y_test_preds[i] = 2
        elif y_test_preds[i] <= point_3:
            y_test_preds[i] = 3
        elif y_test_preds[i] <= point_4:
            y_test_preds[i] = 4
        else:
            print("An error occurred!")
            break
    
        return y_test_preds

In [4]:
df_raw = pd.read_csv('data/survey_results_public.csv', low_memory=False)
schema = pd.read_csv('data/survey_results_schema.csv')

In [5]:
df = df_raw.dropna(subset=['JobSat'], axis=0)
df = df[df.Employment.isin(['Employed full-time', 'Employed part-time'])]
df = df[df.Student == 'No']
df = df.drop(['CareerSat','Respondent','ResumeUpdate','CurrencySymbol','CurrencyDesc','CompTotal','SurveyEase','SurveyLength','SONewContent','WelcomeChange','SOComm','EntTeams','SOVisit1st',
                 'SOVisitFreq', 'SOVisitTo', 'SOFindAnswer', 'SOTimeSaved', 'SOHowMuchTime', 'SOAccount','SOPartFreq', 'SOJobs', 'LanguageWorkedWith', 'LanguageDesireNextYear',
                 'DatabaseWorkedWith', 'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 'WebFrameWorkedWith', 'WebFrameDesireNextYear', 'MiscTechWorkedWith', 
                 'MiscTechDesireNextYear', 'DevEnviron','Student','Ethnicity','JobSeek','MgrIdiot'], axis=1)
df = df.dropna(thresh=45)
df = df.reset_index(drop=True)

In [6]:
EduOther_strings = ['Taken an online course in programming or software development (e.g. a MOOC)','Participated in a fulltime developer training program or bootcamp',
                    'Taken a part-time in-person course in programming or software development','Completed an industry certification program (e.g. MCPD)',
                    'Received on-the-job training in software development','Taught yourself a new language, framework, or tool without taking a formal course',
                    'Participated in online coding competitions (e.g. HackerRank, CodeChef, TopCoder)','Participated in a hackathon','Contributed to open source software',
                    'None of these']

DevType_strings = ['Academic researcher','Data or business analyst','Data scientist or machine learning specialist','Database administrator','Designer','Developer, back-end',
                   'Developer, desktop or enterprise applications','Developer, embedded applications or devices','Developer, front-end','Developer, full-stack',
                   'Developer, game or graphics','Developer, mobile','Developer, QA or test','DevOps specialist','Educator','Engineer, data','Engineer, site reliability',
                   'Engineering manager','Marketing or sales professional','Product manager','Scientist',"Senior Executive (C-Suite, VP, etc.)", 'Student', 'System administrator']

LastInt_strings = ['Write any code','Write code by hand (e.g., on a whiteboard)','Complete a take-home project','Solve a brain-teaser style puzzle','Interview with people in peer roles',
                   'Interview with people in senior / management roles']

WorkChallenge_strings = ['Distracting work environment','Being tasked with non-development work','Meetings','Time spent commuting','Not enough people for the workload',
                         'Toxic work environment','Inadequate access to necessary tools','Lack of support from management','Non-work commitments (parenting, school work, hobbies, etc.)']

JobFactors_strings = ['Diversity of the company or organization',"Languages, frameworks, and other technologies I'd be working with","Industry that I'd be working in",
                      'How widely used or impactful my work output would be',"Specific department or team I'd be working on",'Flex time or a flexible schedule',
                      'Remote work options','Financial performance or funding status of the company or organization','Office environment or company culture',
                      'Opportunities for professional development']

df, dict_EduOther = handle_multi_string_columns(df,"EduOther",EduOther_strings)
df, dict_DevType = handle_multi_string_columns(df,"DevType",DevType_strings)
df, dict_LastInt = handle_multi_string_columns(df,"LastInt",LastInt_strings)
df, dict_WorkChallenge = handle_multi_string_columns(df,"WorkChallenge",WorkChallenge_strings)
df, dict_JobFactors = handle_multi_string_columns(df,"JobFactors",JobFactors_strings)

In [7]:
for n,c in df.items():
        if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()

df.JobSat.cat.set_categories(['Very dissatisfied','Slightly dissatisfied','Neither satisfied nor dissatisfied','Slightly satisfied', 'Very satisfied'], ordered=True, inplace=True)
#df.JobSeek.cat.set_categories(['I am not interested in new job opportunities','I’m not actively looking, but I am open to new opportunities','I am actively looking for a job'], ordered=True, inplace=True)

In [8]:
df_indicator = df.isnull().astype(int).add_suffix('_nan')
df = pd.concat([df, df_indicator], axis=1)

In [9]:
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

fill_mean = lambda col: col.fillna(col.mean())
# Fill the mean
df = df.apply(fill_mean, axis=0)

In [10]:
#Drop columns with all NaN values
df = df.dropna(how='all', axis=0)
df = df.dropna(how='all', axis=1)
#delete columns that add up to 0
df = df.loc[:, (df != 0).any(axis=0)]

In [11]:
X = df.drop(['JobSat'], axis=1)
y = df['JobSat']

X_train, X_test_valid, y_train, y_test_valid = train_test_split(X, y, test_size=0.4, random_state=14)
X_test, X_valid, y_test, y_valid = train_test_split(X_test_valid, y_test_valid, test_size=0.5, random_state=14)

In [12]:
# Grid search for good parameters, I used the mean absolute error as the main measure of quality
param_grid = {'num_leaves': [8,10,20,31],
              'learning_rate': [0.09, 0.1, 0.11, 0.12,0.15],
              'max_bin': [255, 270, 280]}
grid = GridSearchCV(lgm.LGBMRegressor(), param_grid=param_grid,cv=5, return_train_score=True)
X_train_grit = X_train.sample(frac=0.5, random_state=42)
grid.fit(X_train_grit, y_train.loc[X_train_grit.index])
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))
print("test-set score: {:.3f}".format(grid.score(X_test, y_test)))

best mean cross-validation score: 0.158
best parameters: {'learning_rate': 0.1, 'max_bin': 255, 'num_leaves': 8}
test-set score: 0.164


In [13]:
model = lgm.LGBMRegressor(n_estimators=300, 
                          num_leaves=grid.best_params_['num_leaves'],
                          learning_rate=grid.best_params_['learning_rate'],
                          max_bin=grid.best_params_['max_bin'])
model.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_bin=255,
              max_depth=-1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=300, n_jobs=-1, num_leaves=8,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [18]:
#Predict using your model
y_test_preds = model.predict(X_test)
y_train_preds =model.predict(X_train)
#Score using your model
test_r2 = r2_score(y_test, y_test_preds)
train_r2 = r2_score(y_train, y_train_preds)
# train_mad = mad(y_train, classification(y_train, y_train_preds))
# test_mad = mad(y_test, classification(y_train, y_test_preds))

correct = lambda x, multi, add: multi * x + add
corrector = 0 
train_mad = 1
for add in range(-100, 100):
    for multi in range (0, 200):
        new_mad = mad(y_train, np.round(correct(y_train_preds, multi/100, add/100)))
        if new_mad < train_mad:
            train_mad = new_mad
            multi_final = multi / 100
            add_final = add / 100

test_mad = mad(y_test, np.round(correct(y_test_preds, multi_final,add_final)))

print("Multiplier: {}".format(multi_final))
print("Scalar: {}".format(add_final))
print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_r2, test_r2))
print("The mad on the training data was {}.  The mad on the test data was {}.".format(train_mad, test_mad))

Multiplier: 1.36
Scalar: -0.75
The rsquared on the training data was 0.2525598379867674.  The rsquared on the test data was 0.17852236359557638.
The mad on the training data was 0.7955611692529773.  The mad on the test data was 0.8980188372848328.


In [15]:
[np.sum(y_test == 0) , np.sum(y_test == 1), np.sum(y_test == 2), np.sum(y_test == 3), np.sum(y_test == 4)]

[654, 1441, 1012, 3214, 2916]

In [17]:
[np.sum(np.round(correct(y_test_preds, multi_final,add_final)) == 0) , np.sum(np.round(correct(y_test_preds, multi_final,add_final)) == 1), np.sum(np.round(correct(y_test_preds, multi_final,add_final)) == 2), np.sum(np.round(correct(y_test_preds, multi_final,add_final)) == 3), np.sum(np.round(correct(y_test_preds, multi_final,add_final)) == 4)]

[4, 241, 2175, 4723, 2090]

In [19]:
[np.sum(np.round(y_test_preds) == 0) , np.sum(np.round(y_test_preds) == 1), np.sum(np.round(y_test_preds) == 2), np.sum(np.round(y_test_preds) == 3), np.sum(np.round(y_test_preds) == 4)]

[0, 123, 2909, 5849, 356]